# SWR

A flexible framework for safe withdrawal rate experiments.

Framework can generalize to
- Any generator of asset returns
- Any asset allocation strategy based on age, returns etc.
- Any utility function to evaluate suitability of strategy (e.g. total spending, certainty equivalent spending)
- Support a survival table 
- Any optimizer to find optimal parameters for a given withdrawal framework and market simulation


In [3]:
#import sys
#sys.path.append("../")
import pytest
import numpy as np
import pandas as pd

from SWRsimulation.SWRsimulationCE import SWRsimulationCE


In [5]:
# load from pickle
RETURN_FILE = '../data/histretSP'
def load_returns():
    return pd.read_pickle('%s.pickle' % RETURN_FILE)

download_df = load_returns()
return_df = download_df.iloc[:, [0, 3, 16]]
return_df.columns=['stocks', 'bonds', 'cpi']
return_df

,stocks,bonds,cpi
Year,,,
1928,0.438112,0.032196,-0.011561
1929,-0.082979,0.030179,0.005848
1930,-0.251236,0.005398,-0.063953
1931,-0.438375,-0.156808,-0.093168
1932,-0.086424,0.235896,-0.102740
...,...,...,...
2018,-0.042269,-0.027626,0.019102
2019,0.312117,0.153295,0.022851
2020,0.180232,0.104115,0.013620


In [3]:
# should adjust CPI to year-ending also but leave it for now
real_return_df = return_df.copy()
# real_return_df.loc[1948:, 'cpi'] = cpi_test['cpi_fred']
# adjust returns for inflation
real_return_df['stocks'] = (1 + real_return_df['stocks']) / (1 + real_return_df['cpi']) - 1
real_return_df['bonds'] = (1 + real_return_df['bonds']) / (1 + real_return_df['cpi']) - 1
real_return_df.drop('cpi', axis=1, inplace=True)
real_return_df

,stocks,bonds
Year,,
1928,0.454932,0.044268
1929,-0.088311,0.024189
1930,-0.200079,0.074090
1931,-0.380674,-0.070178
1932,0.018184,0.377411
...,...,...
2018,-0.060220,-0.045852
2019,0.282803,0.127529
2020,0.164373,0.089279


In [4]:
# zero returns, zero spending (just check shape)
RETURN = 0.0
# spending
FIXED = 0
VARIABLE = 0.0
NYEARS = 30

s = SWRsimulationCE({
    'simulation': {'returns_df': pd.DataFrame({'stocks': np.zeros(len(real_return_df)), 
                                               'bonds': np.zeros(len(real_return_df))}, 
                                              index=real_return_df.index),
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': 0,
                  },
    'evaluation': {'gamma': 0},
    'visualization': {}    
})
s.simulate()

print(s)

# just simulate 1st year in trials
z = s.latest_simulation[0]['trial']
assert len(z) == 30
assert(z.index[0]) == 1928, "start year == 1928"
assert(z.index[-1]) == 1957, "end year == 1957"

z

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0
2021     0.0    0.0
2022     0.0    0.0

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x132ac2f80>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 0.0,
 'fixed_pct': 0,
 'floor': 0.0,
 'floor_pct': 0,
 'variable': 0.0,
 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1929,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1930,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1931,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1932,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1933,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1934,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1935,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1936,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1937,100.0,0.0,100.0,0.0,100.0,0.5,0.5


In [5]:
# zero returns, spend 2% of starting portfolio per year, check ending value declines to 0.4
RETURN = 0.0
# spending
FIXED = 2.0
VARIABLE = 0.00
NYEARS = 30

returns_df = pd.DataFrame(index=range(1928, 2021), data={'stocks': RETURN, 'bonds': RETURN})


s = SWRsimulationCE({
    'simulation': {'returns_df': pd.DataFrame({'stocks': np.zeros(len(real_return_df)), 
                                               'bonds': np.zeros(len(real_return_df))}, 
                                              index=real_return_df.index),
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': 0,
                  },
    'evaluation': {'gamma': 0},
    'visualization': {}    
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert(z['end_port'].iloc[-1]) == 40, "ending port value == 40"

z

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0
2021     0.0    0.0
2022     0.0    0.0

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x132ac2500>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 2.0,
 'fixed_pct': 2.0,
 'floor': 0.0,
 'floor_pct': 0,
 'variable': 0.0,
 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.0,100.0,2.0,98.0,0.5,0.5
1929,98.0,0.0,98.0,2.0,96.0,0.5,0.5
1930,96.0,0.0,96.0,2.0,94.0,0.5,0.5
1931,94.0,0.0,94.0,2.0,92.0,0.5,0.5
1932,92.0,0.0,92.0,2.0,90.0,0.5,0.5
1933,90.0,0.0,90.0,2.0,88.0,0.5,0.5
1934,88.0,0.0,88.0,2.0,86.0,0.5,0.5
1935,86.0,0.0,86.0,2.0,84.0,0.5,0.5
1936,84.0,0.0,84.0,2.0,82.0,0.5,0.5
1937,82.0,0.0,82.0,2.0,80.0,0.5,0.5


In [6]:
# zero returns, spend 2% of current portfolio per year, check ending value declines to 0.98 ** 30
RETURN = 0.0
FIXED = 0
VARIABLE = 2.0
NYEARS = 30

s = SWRsimulationCE({
    'simulation': {'returns_df': pd.DataFrame({'stocks': np.zeros(len(real_return_df)), 
                                               'bonds': np.zeros(len(real_return_df))}, 
                                              index=real_return_df.index),
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': 0,
                  },
    'evaluation': {'gamma': 0},
    'visualization': {}    
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert z['end_port'].iloc[-1] == pytest.approx(100 * ((1 - VARIABLE/100) ** NYEARS), 0.000001)
z

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0
2021     0.0    0.0
2022     0.0    0.0

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x132ac33e0>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 0.0,
 'fixed_pct': 0,
 'floor': 0.0,
 'floor_pct': 0,
 'variable': 0.02,
 'variable_pct': 2.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.000000,0.0,100.000000,2.000000,98.000000,0.5,0.5
1929,98.000000,0.0,98.000000,1.960000,96.040000,0.5,0.5
1930,96.040000,0.0,96.040000,1.920800,94.119200,0.5,0.5
1931,94.119200,0.0,94.119200,1.882384,92.236816,0.5,0.5
1932,92.236816,0.0,92.236816,1.844736,90.392080,0.5,0.5
1933,90.392080,0.0,90.392080,1.807842,88.584238,0.5,0.5
1934,88.584238,0.0,88.584238,1.771685,86.812553,0.5,0.5
1935,86.812553,0.0,86.812553,1.736251,85.076302,0.5,0.5
1936,85.076302,0.0,85.076302,1.701526,83.374776,0.5,0.5
1937,83.374776,0.0,83.374776,1.667496,81.707281,0.5,0.5


In [7]:
# 4% real return, spend fixed 4% of starting, assert ending value unchanged
RETURN = 0.04
FIXED = 4 
FLOOR_PCT = 0.0
VARIABLE = 0.0
NYEARS = 30

returns_df = pd.DataFrame(index=real_return_df.index, data={'stocks': RETURN, 'bonds': RETURN})

s = SWRsimulationCE({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR_PCT},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert(z['end_port'].iloc[-1]) == 100, "end port value correct"
z


Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928    0.04   0.04
1929    0.04   0.04
1930    0.04   0.04
1931    0.04   0.04
1932    0.04   0.04
...      ...    ...
2018    0.04   0.04
2019    0.04   0.04
2020    0.04   0.04
2021    0.04   0.04
2022    0.04   0.04

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x132ac3990>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 4.0,
 'fixed_pct': 4,
 'floor': 0.0,
 'floor_pct': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1929,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1930,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1931,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1932,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1933,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1934,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1935,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1936,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1937,100.0,0.04,104.0,4.0,100.0,0.5,0.5


In [8]:
# return 0.02% variable spending 0.02/1.02, check final value unchanged
RETURN = 0.02
FIXED = 0.0
FLOOR = 0.0
VARIABLE = 0.02/1.02*100
NYEARS = 30

returns_df = pd.DataFrame(index=real_return_df.index, data={'stocks': RETURN, 'bonds': RETURN})

s = SWRsimulationCE({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert (z['start_port'].iloc[0]) == 100, "start port value == 100"
assert (z['end_port'].iloc[-1]) == 100, "end port value correct"
z


Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928    0.02   0.02
1929    0.02   0.02
1930    0.02   0.02
1931    0.02   0.02
1932    0.02   0.02
...      ...    ...
2018    0.02   0.02
2019    0.02   0.02
2020    0.02   0.02
2021    0.02   0.02
2022    0.02   0.02

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x132ac3840>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 0.0,
 'fixed_pct': 0.0,
 'floor': 0.0,
 'floor_pct': 0.0,
 'variable': 0.0196078431372549,
 'variable_pct': 1.9607843137254901}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1929,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1930,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1931,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1932,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1933,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1934,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1935,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1936,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1937,100.0,0.02,102.0,2.0,100.0,0.5,0.5


In [9]:
# check values per appendix of Bengen paper https://www.retailinvestor.org/pdf/Bengen1.pdf
# nominal return 10% for stocks, 5% for bonds
# inflation 3%
# fixed spending of 4% of orig port
STOCK_RETURN = (1.1 / 1.03) - 1
BOND_RETURN = (1.05 / 1.03) - 1
VARIABLE = 0.0
FIXED = 4.0
FLOOR = 0.0
NYEARS = 30

returns_df = pd.DataFrame(index=real_return_df.index, data={'stocks': STOCK_RETURN, 'bonds': BOND_RETURN})

s = SWRsimulationCE({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

# match figures in appendix
# example uses nominal vals with 3% inflation, we use real vals
assert z.iloc[0]['before_spend'] * 1.03 == pytest.approx(107.5, 0.000001)
assert z.iloc[0]['spend'] * 1.03 == 4.12, "spend does not match Bengen"
assert z.iloc[0]['end_port'] * 1.03 == pytest.approx(103.38, 0.000001), "ending port does not match Bengen"

z

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.067961  0.019417
1929  0.067961  0.019417
1930  0.067961  0.019417
1931  0.067961  0.019417
1932  0.067961  0.019417
...        ...       ...
2018  0.067961  0.019417
2019  0.067961  0.019417
2020  0.067961  0.019417
2021  0.067961  0.019417
2022  0.067961  0.019417

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x132ac3d10>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 4.0,
 'fixed_pct': 4.0,
 'floor': 0.0,
 'floor_pct': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.000000,0.043689,104.368932,4.0,100.368932,0.5,0.5
1929,100.368932,0.043689,104.753982,4.0,100.753982,0.5,0.5
1930,100.753982,0.043689,105.155855,4.0,101.155855,0.5,0.5
1931,101.155855,0.043689,105.575286,4.0,101.575286,0.5,0.5
1932,101.575286,0.043689,106.013041,4.0,102.013041,0.5,0.5
1933,102.013041,0.043689,106.469922,4.0,102.469922,0.5,0.5
1934,102.469922,0.043689,106.946763,4.0,102.946763,0.5,0.5
1935,102.946763,0.043689,107.444437,4.0,103.444437,0.5,0.5
1936,103.444437,0.043689,107.963854,4.0,103.963854,0.5,0.5
1937,103.963854,0.043689,108.505964,4.0,104.505964,0.5,0.5


In [10]:
VARIABLE = 0.0
FIXED = 4.0
NYEARS = 30
FLOOR = 0.0

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
#    'allocation': {},  # no args, default equal weight
    'allocation': {'asset_weights': np.array([0.55, 0.45])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate()

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.454932  0.044268
1929 -0.088311  0.024189
1930 -0.200079  0.074090
1931 -0.380674 -0.070178
1932  0.018184  0.377411
...        ...       ...
2018 -0.060220 -0.045852
2019  0.282803  0.127529
2020  0.164373  0.089279
2021  0.199522 -0.057578
2022 -0.229552 -0.196470

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x132ac1ee0>}

Allocation:
{'asset_weights': array([0.55, 0.45])}

Withdrawal:
{'fixed': 4.0,
 'fixed_pct': 4.0,
 'floor': 0.0,
 'floor_pct': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


In [94]:
z

[{'trial':       start_port  port_return  before_spend  spend    end_port  alloc_0  \
  1928  100.000000     0.270133    127.013296    4.5  122.513296     0.55   
  1929  122.513296    -0.037686    117.896264    4.5  113.396264     0.55   
  1930  113.396264    -0.076703    104.698441    4.5  100.198441     0.55   
  1931  100.198441    -0.240951     76.055509    4.5   71.555509     0.55   
  1932   71.555509     0.179836     84.423782    4.5   79.923782     0.55   
  1933   79.923782     0.323153    105.751379    4.5  101.251379     0.55   
  1934  101.251379     0.062045    107.533545    4.5  103.033545     0.55   
  1935  103.033545     0.278784    131.757700    4.5  127.257700     0.55   
  1936  127.257700     0.209389    153.904000    4.5  149.404000     0.55   
  1937  149.404000    -0.236052    114.136896    4.5  109.636896     0.55   
  1938  109.636896     0.236977    135.618270    4.5  131.118270     0.55   
  1939  131.118270     0.029888    135.037069    4.5  130.537069   

In [95]:
import plotly.express as px
from plotly import graph_objects as go

In [96]:
years_survived = pd.DataFrame(data={'nyears': [30 - len(np.where(y['trial']['spend']==0.0)[0]) for y in z]},
                              index=range(1928,1994)).reset_index()

px.bar(years_survived, x="index", y="nyears", color="nyears",
              hover_name="index", color_continuous_scale="spectral")


In [80]:
portval_df = pd.DataFrame(data=np.hstack([(np.ones(66).reshape(66, 1) * 100), np.array([y['trial']['end_port'].values for y in z])])) \
    .transpose()
portval_df.columns=range(1928,1994)
portval_df['mean'] = portval_df.mean(axis=1)
portval_df

col_list = list(portval_df.columns)
portval_df.reset_index(inplace=True)
portval_melt = pd.melt(portval_df, id_vars=['index'], value_vars=col_list)
portval_melt.columns=['ret_year', 'start_year', 'portval']
portval_melt

,ret_year,start_year,portval
0,0,1928,100.000000
1,1,1928,119.959978
2,2,1928,111.113946
3,3,1928,99.114379
4,4,1928,71.771384
...,...,...,...
2072,26,mean,141.695244
2073,27,mean,143.134444
2074,28,mean,145.983444
2075,29,mean,150.671990


In [81]:
portval_df

,index,1928,1929,1930,1931,1932,1933,1934,1935,1936,...,1985,1986,1987,1988,1989,1990,1991,1992,1993,mean
0,0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,1,119.959978,91.793904,88.700550,72.457362,114.779755,125.478539,102.189845,121.255268,114.925556,...,117.881050,113.686983,94.633161,105.923947,113.142011,90.694454,115.356117,101.736927,105.171325,100.953641
2,2,111.113946,81.011393,63.705107,81.789251,144.762946,129.500251,124.020063,140.416055,84.524581,...,134.909461,108.269934,99.970838,120.140682,103.270629,104.156322,118.127574,107.084920,97.434828,101.554928
3,3,99.114379,57.749288,71.305821,101.717419,150.171178,158.500889,143.731750,104.381141,98.683496,...,129.414561,115.097285,113.107558,109.967969,119.292519,106.173258,125.142713,99.298633,117.271210,102.279374
4,4,71.771384,64.171956,88.038793,104.030744,184.599023,185.083073,106.963998,123.040879,97.080958,...,138.551739,130.978248,103.237660,127.353177,122.329170,111.972485,116.886573,119.610104,124.091464,103.383851
5,5,80.967587,78.730630,89.368645,126.344295,216.381405,139.175805,126.209183,122.276913,90.416590,...,158.687812,120.338918,119.252838,130.932868,129.771667,104.059026,141.681254,126.666104,144.430013,105.288063
6,6,100.645325,79.391241,107.832623,146.519098,163.556545,165.722337,125.554296,115.180583,74.056870,...,146.855434,139.835249,122.286815,139.250475,121.395083,125.583933,150.961897,147.530376,162.925292,107.253274
7,7,102.881567,95.235624,124.318872,109.135283,195.629381,166.427803,118.401778,95.709576,71.192726,...,171.749498,144.255848,129.725004,130.627259,147.338995,133.242060,176.787189,166.530009,170.903675,109.492538
8,8,124.893399,109.211851,91.841776,128.872626,197.364483,158.574545,98.526067,93.469912,75.592847,...,178.320137,153.928579,121.349635,158.924475,157.189905,155.449093,200.546199,174.795539,160.551300,111.715675
9,9,144.779103,80.073726,107.659256,128.309436,188.980842,133.651626,96.367633,100.811460,78.351303,...,191.457657,144.923446,147.281962,169.943135,184.286910,175.736910,211.521407,164.321278,149.911904,112.985457


In [82]:
fig = go.Figure()
for year in range(1928,1994):
    
    fig.add_trace(go.Scatter(x=portval_df['index'], 
                             y=portval_df[year],
                             mode='lines',
                             name=str(year),
                             line={'width': 1},
                            ),
                 )

fig.add_trace(go.Scatter(x=portval_df['index'], 
                         y=portval_df['mean'],
                         mode='lines',
                         name='Mean',
                         line={'width': 3, 'color': 'black'},
                        ),
             )
    
fig.update_layout(showlegend=False,
                  plot_bgcolor="white",
                  title="Retirement Outcomes, 1928-1994",
                  xaxis=dict(title="Retirement Year", linecolor='black', mirror=True, ticks='inside',),
                  yaxis=dict(title="Portfolio Value", linecolor='black', mirror=True, ticks='inside'),
                 )
fig.show()


In [83]:
fig = px.line(portval_melt, x="ret_year", y="portval", color="start_year",
              hover_name="start_year")
fig.show()

In [88]:
VARIABLE = 0.0
FIXED = 4.0
FLOOR = 0.0
NYEARS = 30
NTRIALS = 10000

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'montecarlo': 10000,
                   'montecarlo_replacement': False,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=False)

Simulation:
{'montecarlo': 10000,
 'montecarlo_replacement': False,
 'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.454932  0.044268
1929 -0.088311  0.024189
1930 -0.200079  0.074090
1931 -0.380674 -0.070178
1932  0.018184  0.377411
...        ...       ...
2018 -0.060220 -0.045852
2019  0.282803  0.127529
2020  0.164373  0.089279
2021  0.199522 -0.057578
2022 -0.229552 -0.196470

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.montecarlo_trials at 0x13c902f80>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 4.0,
 'fixed_pct': 4.0,
 'floor': 0.0,
 'floor_pct': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


In [85]:
c, bins = np.histogram([y['exhaustion'] for y in z], bins=list(range(31)))
pct_exhausted = np.sum(c[:29])/np.sum(c) * 100
print ("%.2f%% of portfolios exhausted before final year" % pct_exhausted)
bins += 1
fig = go.Figure([go.Bar(x=bins, y=c)])
fig.update_layout(showlegend=False,
                  plot_bgcolor="white",
                  title="Histogram of years to exhaustion",
                  xaxis=dict(title="Retirement Year", 
                             linecolor='black', mirror=True, ticks='inside',),
                  yaxis=dict(title="Number of Portfolios Exhausted (log scale)", 
                             linecolor='black', mirror=True, ticks='inside',
                             type="log"),
                 )

fig


4.74% of portfolios exhausted before final year


In [55]:
# Bengen 4% rule - ending val of 1st cohort = 189.255136
FIXED = 4.0
VARIABLE = 0.0
FLOOR = 4.0
NYEARS = 30

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=True)
assert z[0]['trial'].iloc[0]['spend'] == 4.0, "bad value: cohort 0 year 0 spend"
assert z[0]['trial'].iloc[0]['end_port'] == pytest.approx(120.955061, 0.000001), "bad value: cohort 0 year 0 end port"
assert z[0]['trial'].iloc[-1]['spend'] == 4.0, "bad value: cohort 0 final year spend"
assert z[0]['trial'].iloc[-1]['end_port'] == pytest.approx(189.255136, 0.000001), "bad value: cohort 0 final year end port"
z[0]['trial']

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.454932  0.044268
1929 -0.088311  0.024189
1930 -0.200079  0.074090
1931 -0.380674 -0.070178
1932  0.018184  0.377411
...        ...       ...
2018 -0.060220 -0.045852
2019  0.282803  0.127529
2020  0.164373  0.089279
2021  0.199522 -0.057578
2022 -0.229552 -0.196470

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x13c901770>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 4.0,
 'fixed_pct': 4.0,
 'floor': 4.0,
 'floor_pct': 4.0,
 'variable': 0.0,
 'variable_pct': 0.0}


AssertionError: bad value: cohort 0 year 0 end port

In [ ]:
print('initial', np.max([s.withdrawal['fixed_pct'] + s.withdrawal['variable_pct'], s.withdrawal['floor_pct']]))
print('mean', np.mean([y['mean_spend'] for y in s.latest_simulation]))
print('worst', np.min([y['min_spend'] for y in s.latest_simulation]))
print('exhaustion pct', np.sum(np.where([y['exhaustion'] < 30 for y in s.latest_simulation], 1, 0))/30)


initial 4.0
mean 3.998027271709438
worst 0.09399798468684553
exhaustion pct 0.03333333333333333


In [ ]:
# relaxed 4%/5% rule 
FIXED = -1
VARIABLE = 5.0
FLOOR = 4.0
NYEARS = 30

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=True)
assert z[0]['trial'].iloc[0]['spend'] == pytest.approx(5.247753, 0.000001), "bad value: cohort 0 year 0 spend"
assert z[0]['trial'].iloc[0]['end_port'] == pytest.approx(119.707308, 0.000001), "bad value: cohort 0 year 0 end port"
assert z[0]['trial'].iloc[-1]['spend'] == pytest.approx(5.690874, 0.000001), "bad value: cohort 0 final year spend"
assert z[0]['trial'].iloc[-1]['end_port'] == pytest.approx(128.126609, 0.000001), "bad value: cohort 0 final year end port"
z[0]['trial']


Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.454932  0.044268
1929 -0.088311  0.024189
1930 -0.200079  0.074090
1931 -0.380674 -0.070178
1932  0.018184  0.377411
...        ...       ...
2018 -0.060220 -0.045852
2019  0.282803  0.127529
2020  0.164373  0.089279
2021  0.199522 -0.057578
2022 -0.229552 -0.196470

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x13ed91070>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': -1.0,
 'fixed_pct': -1,
 'floor': 4.0,
 'floor_pct': 4.0,
 'variable': 0.05,
 'variable_pct': 5.0}


AssertionError: bad value: cohort 0 year 0 spend

In [ ]:
print('initial', np.max([s.withdrawal['fixed_pct'] + s.withdrawal['variable_pct'], s.withdrawal['floor_pct']]))
print('mean', np.mean([y['mean_spend'] for y in s.latest_simulation]))
print('worst', np.min([y['min_spend'] for y in s.latest_simulation]))
print('exhaustion pct', np.sum(np.where([y['exhaustion'] < 30 for y in s.latest_simulation], 1, 0))/30)


initial 4.0
mean 5.753933550893551
worst 0.09399798468684553
exhaustion pct 0.03333333333333333


In [97]:
# higher risk aversion rule 
FIXED = 3.5
VARIABLE = 1.1
FLOOR = 3.8
NYEARS = 30
STOCK_PCT = 0.73
BOND_PCT = 0.27

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([STOCK_PCT, BOND_PCT])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=True)
assert z[0]['trial'].iloc[0]['spend'] == pytest.approx(4.978399, 0.000001), "bad value: cohort 0 year 0 spend"
assert z[0]['trial'].iloc[0]['end_port'] == pytest.approx(129.421552, 0.000001), "bad value: cohort 0 year 0 end port"
assert z[0]['trial'].iloc[-1]['spend'] == pytest.approx(5.068669, 0.000001), "bad value: cohort 0 final year spend"
assert z[0]['trial'].iloc[-1]['end_port'] == pytest.approx(137.537621, 0.000001), "bad value: cohort 0 final year end port"
z[0]['trial']


Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.454932  0.044268
1929 -0.088311  0.024189
1930 -0.200079  0.074090
1931 -0.380674 -0.070178
1932  0.018184  0.377411
...        ...       ...
2018 -0.060220 -0.045852
2019  0.282803  0.127529
2020  0.164373  0.089279
2021  0.199522 -0.057578
2022 -0.229552 -0.196470

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x14f2fba70>}

Allocation:
{'asset_weights': array([0.73, 0.27])}

Withdrawal:
{'fixed': 3.5000000000000004,
 'fixed_pct': 3.5,
 'floor': 3.8,
 'floor_pct': 3.8,
 'variable': 0.011000000000000001,
 'variable_pct': 1.1}


AssertionError: bad value: cohort 0 year 0 spend

In [98]:
print('initial', np.max([s.withdrawal['fixed_pct'] + s.withdrawal['variable_pct'], s.withdrawal['floor_pct']]))
print('mean', np.mean([y['mean_spend'] for y in s.latest_simulation]))
print('worst', np.min([y['min_spend'] for y in s.latest_simulation]))
print('exhaustion pct', np.sum(np.where([y['exhaustion'] < 30 for y in s.latest_simulation], 1, 0))/30)


initial 4.6
mean 5.291180692598103
worst 0.0
exhaustion pct 0.03333333333333333


In [ ]:
# lower risk aversion rule
FIXED = 0.7
VARIABLE = 5.8
FLOOR = 3.4
NYEARS = 30
STOCK_PCT = 0.89
BOND_PCT = 0.11

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([STOCK_PCT, BOND_PCT])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=True)
assert z[0]['trial'].iloc[0]['spend'] == pytest.approx(8.876278, 0.000001), "bad value: cohort 0 year 0 spend"
assert z[0]['trial'].iloc[0]['end_port'] == pytest.approx(132.094032, 0.000001), "bad value: cohort 0 year 0 end port"
assert z[0]['trial'].iloc[-1]['spend'] == pytest.approx(5.135414, 0.000001), "bad value: cohort 0 final year spend"
assert z[0]['trial'].iloc[-1]['end_port'] == pytest.approx(71.337240, 0.000001), "bad value: cohort 0 final year end port"
z[0]['trial']



Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.454932  0.044268
1929 -0.088311  0.024189
1930 -0.200079  0.074090
1931 -0.380674 -0.070178
1932  0.018184  0.377411
...        ...       ...
2018 -0.060220 -0.045852
2019  0.282803  0.127529
2020  0.164373  0.089279
2021  0.199522 -0.057578
2022 -0.229552 -0.196470

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x13ecb5540>}

Allocation:
{'asset_weights': array([0.89, 0.11])}

Withdrawal:
{'fixed': 0.7,
 'fixed_pct': 0.7,
 'floor': 3.4000000000000004,
 'floor_pct': 3.4,
 'variable': 0.057999999999999996,
 'variable_pct': 5.8}


AssertionError: bad value: cohort 0 year 0 spend

In [ ]:
print('initial', np.max([s.withdrawal['fixed_pct'] + s.withdrawal['variable_pct'], s.withdrawal['floor_pct']]))
print('mean', np.mean([y['mean_spend'] for y in s.latest_simulation]))
print('worst', np.min([y['min_spend'] for y in s.latest_simulation]))
print('exhaustion pct', np.sum(np.where([y['exhaustion'] < 30 for y in s.latest_simulation], 1, 0))/30)


initial 6.5
mean 7.539549222690148
worst 1.896026991374592
exhaustion pct 0.03333333333333333
